<!--
tags: [tutorial]
description: Conversation patterns
-->

# Conversation Patterns

In the [last chapter](./code-executors) we used two-agent conversation, which
can be started by the `initiate_chat` method.
Two-agent chat is a useful conversation pattern but AutoGen offers more.
In this chapter, we will first dig a little bit more into the two-agent 
chat pattern and chat result, 
then we will show you several conversation patterns that involve 
more than two agents.


In [2]:
import os
from autogen import ConversableAgent, GroupChat, GroupChatManager

## Two-Agent Chat and Chat Result

Two-agent chat is the simplest from of conversation pattern. We have already
seen multiple examples of two-agent chat in the previous chapters
but there are still some details that we haven't covered yet.
The following figure illustrates how two-agent chat works.

```{=mdx}
![Two-agent chat](./assets/two-agent-chat.png)
```

Besides the conversation itself, there is also a summarizer that can be used
to produce a summary of the conversation. The summarizer is optional and by
default the summary is the last message of the conversation (i.e., `last_msg`).
This can be changed by setting the `summary_method` parameter of the `initiate_chat`.
The example below shows how to use an LLM-based summarizer.

In [15]:
agent_a = ConversableAgent(
    name="Agent A",
    system_message="You are a student willing to learn.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)
agent_b = ConversableAgent(
    name="Agent B",
    system_message="You are a math teacher.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = agent_a.initiate_chat(
    agent_b,
    message="What is triangle inequality?",
    summary_method="reflection_with_llm",
    max_turns=2,
)

Agent A (to Agent B):

What is triangle inequality?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent B (to Agent A):

Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:

a + b > c
a + c > b
b + c > a

Each of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


Agent A (to Agent B):

Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangle inequality theorem?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Agent B (to Agent A):

Absolutely! Here's an example:

Suppose you're given three line segments with lengths 10, 7, and 3 units. The question is: "Can these three line segments form a triangle?"

To answer this, you would use the triangle inequality theorem. Adding any two side lengths together should be greater than the third:

- For sides 10 and 7: 10 + 7 = 17, which is larger than 3.
- For sides 10 and 3: 10 + 3 = 13, which is larger than 7.
- For sides 7 and 3: 7 + 3 = 10, which is equal to the length of the third side (10), but not greater.

So, these three lines cannot form a t

Let's see what the summary looks like.
The summary is stored in the `chat_result` object of type `ChatResult` 
that was returned by the `initiate_chat` method.

In [16]:
print(chat_result.summary)

The triangle inequality theorem states that in a triangle, the sum of the lengths of any two sides must always be greater than the length of the third side. This principle is significant in geometry and is used in solving problems or proving theorems. For instance, if given three line segments, you can determine if they can form a triangle using this theorem.


In the above example, the summarizer is set to `reflection_with_llm` which
takes a list of messages from the conversation and summarize using a call to
an LLM.
The summarizer first tries to use the recipient's LLM, if it is not available
then it uses the sender's LLM. In this case the recipient is "Agent B" and
the sender is "Agent A".
The input prompt for the LLM is the following default prompt:

In [4]:
print(ConversableAgent.DEFAULT_summary_prompt)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


You can also use a custom prompt by setting the `summary_prompt` argument
of `initiate_chat`.

There are some other useful information in the `chat_result` object, including
the conversation history, human input, and token cost.

In [5]:
# Get the chat history.
print(chat_result.chat_history)

[{'content': 'What is triangle inequality?', 'role': 'assistant'}, {'content': 'Triangle inequality theorem is a fundamental principle in geometry that states that the sum of the lengths of any two sides of a triangle must always be greater than the length of the third side. In a triangle with sides of lengths a, b, and c, the theorem can be written as:\n\na + b > c\na + c > b\nb + c > a\n\nEach of these represents the condition for one specific side (a, b, or c). All must be true for a triangle to exist.', 'role': 'user'}, {'content': 'Thank you for the explanation. This theorem helps in understanding the basic properties of a triangle. It can also be useful when solving geometric problems or proving other mathematical theorems. Can you give me an example of how we can use the triangle inequality theorem?', 'role': 'assistant'}, {'content': 'Absolutely! Here\'s an example:\n\nSuppose you\'re given three line segments with lengths 10, 7, and 3 units. The question is: "Can these three l

In [6]:
# Get the cost of the chat.
print(chat_result.cost)

({'total_cost': 0.04521, 'gpt-4-0613': {'cost': 0.04521, 'prompt_tokens': 709, 'completion_tokens': 399, 'total_tokens': 1108}}, {'total_cost': 0})


## Sequential Chats

The name of this pattern is self-explanatory -- it is a sequence of
chats between two agents, chained together by a mechanism called *carryover*,
which brings the summary of the previous chat to the initial context
of the next chat.

This pattern is useful for complex task that can be broken down into 
interdependent sub-tasks. 
The figure below illustrate how this pattern works.

```{=mdx}
![initiate_chats](./assets/sequential-two-agent-chat.png)
```

In this pattern, the a pair of agents first start a conversation,
then the summary of the conversation is inserted into the initial
message of the next conversation. The previous summary is called a *carryover*.
Carryover accumulates as the conversation goes on, 
so each subsequent conversation is started with all the carryovers 
from previous conversations.

The figure above shows distinct recipient agents for all the chats, however,
the recipient agents in the sequence are allowed to repeat.

To illustrate this pattern, let's consider a simple example of arithmetic
operator agents. One agent 
(called the "Number_Agent") is responsible for coming up with a number,
and other agents are responsible for performing a specific arithmetic
operation on the number, e.g., add 1, multiply by 2, etc..

In [4]:
# The Number Agent that always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Adder Agent that adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Multiplier Agent that multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Subtracter Agent that subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

# The Divider Agent that divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
    human_input_mode="NEVER",
)

The Number Agent chats with the first operator agent, then the second
operator agent, and so on.
In the end we will have the result of the arithmetic operations.

In [7]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
14

With the following carryover: 


********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
These are my numbers

With the following carryover: 
16

********************************************************************************
Number_Agent (to Mul

First thing to note is that the `initiate_chats` method takes a list of
dictionaries, each dictionary contains the arguments for the `initiate_chat`
method.

Second, each chat in the sequence has a maximum round of 2, as specified with
the setting `max_turns=2`, 
which means each arithmetic operation is performed twice.
So you can see in the first chat the number 14 becomes 15 and then 16,
in the second chat the number 16 becomes 32 and then 64, and so on.

Third, the carryover accumulates as the chats go on. In the second chat,
the carryover is the summary of the first chat, which is "16".
In the third chat, the carryover is the summary of the first and second
chat, which is the list "16" and "64", and both numbers are operated upon.
In the forth and last chat, the carryover is the summary of all previous
chats, which is the list "16", "64", "14" and "62", and all of these
numbers are operated upon.

The final note is that the `initiate_chats` method returns a list of
`ChatResult` objects, one for each chat in the sequence.

In [8]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  14
62
Fourth Chat Summary:  4
16
3.5
15.5


Besides calling `initiate_chats` from the same sender agent, you can also
call a high-level function `autogen.agentchat.initiate_chats` to start
a sequence of two-agent chats with different sender agents. 
This function allows you to specify the
sender agent for each chat in the sequence.

## Group Chat

So far we have only seen conversation patterns that involve two agents or 
a sequence of two-agent chats. AutoGen provides a more general conversation
pattern called group chat, which involves more than two agents.
The core idea of group chat is that all agents contribute to a single
conversation thread and share the same context.
This is useful for tasks that require collaboration among multiple agents.

The figure below illustrates how group chat works.

```{=mdx}
![group_chat](./assets/group-chat.png)
```

A group chat is orchestrated by a special
agent type `GroupChatManager`. In the first step of the group chat,
the Group Chat Manager selects an agent to speak. Then, the selected agent
speaks and the message is sent back to the Group Chat Manager,
who **broadcasts** the message to all other agents in the group.
This process repeats until the conversation stops.

The Group Chat Manager can use several strategies to select the next agent.
Currently, the following strategies are supported:

1. `round_robin`: The Group Chat Manager selects agents in a round-robin
   fashion based on the order of the agents provided.
2. `random`: The Group Chat Manager selects agents randomly.
3. `manual`: The Group Chat Manager selects agents by asking for human input.
4. `auto`: The default strategy, which selects agents using the Group Chat
   Manager's LLM.

To illustrate this pattern, let's consider a simple example of a group chat
among the same arithmetic operator agents as in the previous example,
with the objective of turning a number into a specific target number
using a sequence of arithmetic operations powered by the agents.

In this example, we use the `auto` strategy to select the next agent.
To help the Group Chat Manager select the next agent, we also set the
`description` of the agents. Without the `description`, the Group Chat Manager
will use the agents' `system_message`, which may be not be the best choice.

In [5]:
# The `description` attribute is a string that describes the agent.
# It can also be set in `ConversableAgent` constructor.
adder_agent.description = "Add 1 to each input number."
multiplier_agent.description = "Multiply each input number by 2."
subtracter_agent.description = "Subtract 1 from each input number."
divider_agent.description = "Divide each input number by 2."
number_agent.description = "Return the numbers given."

We first create a `GroupChat` object and provide the list of agents.
If we were to use the `round_robin` strategy, this list would specify the order
of the agents to be selected.
We also initialize the group chat with an empty message list and a maximum
round of 6, which means there will be at most 6 iteratiosn of selecting speaker,
agent speaks and broadcasting message.

In [10]:
group_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
)

Now we create a `GroupChatManager` object and provide the `GroupChat` object as
input. We also need to specify the `llm_config` of the Group Chat Manager
so it can use the LLM to select the next agent (the `auto` strategy).

In [11]:
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

Finally, we have the Number Agent from before to start a two-agent chat with
the Group Chat Manager, which runs the group chat internally and terminates
the two-agent chat when the internal group chat is done.
Because the Number Agent is selected to speak by us, it counts as the 
first round of the group chat.

In [12]:
chat_result = number_agent.initiate_chat(
    group_chat_manager,
    message="My number is 3, I want to turn it into 13.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

14

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

13

--------------------------------------------------------------------------------
Number_Agent (to chat_manager):

13

--------------------------------------------------------------------------------


You can see that the Number Agent is selected to speak first, then the
Group Chat Manager selects the Multiplier Agent to speak, then the Adder
Agent, and so on. The number is operated upon by each agent in the group
chat, and the final result is 13.

We can take a look at the summary of the group chat, provided by the 
`ChatResult` object returned by the `initiate_chat` method.

In [13]:
print(chat_result.summary)

The agents cooperatively manipulated the initial number (3) through multipliying, adding, and subtracting operations to reach the target number (13).


### Send Introductions

In the previous example, we set the `description` of the agents to help the
Group Chat Manager select the next agent. This only helps the Group Chat Manager,
however, does not help the participating agents to know about each other.
Sometimes it is useful have each agent introduce themselves to other
agents in the group chat. This can be done by setting the `send_introductions=True`.

In [6]:
group_chat_with_introductions = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    messages=[],
    max_round=6,
    send_introductions=True,
)

Under the hood, the Group Chat Manager sends a message containing the 
agents' names and descriptions to all agents in the group chat before the
group chat starts.

### Group Chat in a Sequential Chat

Group chat can also be used as a part of a sequential chat.
In this case, the Group Chat Manager is treated as a regular agent
in the sequence of two-agent chats.

In [7]:
# Let's use the group chat with introduction messages created above.
group_chat_manager_with_intros = GroupChatManager(
    groupchat=group_chat_with_introductions,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# Start a sequence of two-agent chats between the number agent and
# the group chat manager.
chat_result = number_agent.initiate_chats(
    [
        {
            "recipient": group_chat_manager_with_intros,
            "message": "My number is 3, I want to turn it into 13.",
        },
        {
            "recipient": group_chat_manager_with_intros,
            "message": "Turn this number to 32.",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
My number is 3, I want to turn it into 13.

With the following carryover: 


********************************************************************************
Number_Agent (to chat_manager):

My number is 3, I want to turn it into 13.

--------------------------------------------------------------------------------


/Users/ekzhu/autogen/autogen/agentchat/chat.py:46: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Multiplier_Agent (to chat_manager):

6

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

14

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

13

--------------------------------------------------------------------------------
Number_Agent (to chat_manager):

Your number is 13.

--------------------------------------------------------------------------------

********************************************************************************
Start a new chat with the following message: 
Turn this number to 32.

With the following carryover: 
Your number is 13.

********************************************************************************
Number_Agent (to chat_manager):

Turn this number to 32.
Context: 
Your number is 13.

-------------

In the above example, the Group Chat Manager runs the group chat two times.
In the first time the number 3 becomes 13, and the last message of this group chat
is being used as the carryover for the next group chat, which starts from 13.

You can also see from the warning message that the Group Chat Manager's
history is being cleared after the first group chat, which is the default.
To keep the history of the Group Chat Manager, 
you can set the `clear_history=False` for the first chat.

### Constrained Speaker Selection

Group chat is a powerful conversation pattern, but it can be hard to control
if the number of participating agents is large.
AutoGen provides a way to constrain the selection of the next speaker
by using the `allowed_or_disallowed_speaker_transitions` argument of the 
`GroupChat` class.

The `allowed_or_disallowed_speaker_transitions` argument is a dictionary
that maps a given agent to a list of agents that can (or cannot) 
be selected to speak next. The `speaker_transitions_type` argument specifies
whether the transitions are allowed or disallowed.

Here is an example:

In [17]:
allowed_transitions = {
    number_agent: [adder_agent, number_agent],
    adder_agent: [multiplier_agent, number_agent],
    subtracter_agent: [divider_agent, number_agent],
    multiplier_agent: [subtracter_agent, number_agent],
    divider_agent: [adder_agent, number_agent],
}

In this example, the allowed transitions are specified for each agent.
The Number Agent can be followed by the Adder Agent and the Number Agent, 
the Adder Agent can be followed by the Multiplier Agent and the Number Agent,
and so on.
Let's put this into the group chat and see how it works.
The `speaker_transitions_type` is set to `allowed` so the transitions are
positive constraints.

In [21]:
constrained_graph_chat = GroupChat(
    agents=[adder_agent, multiplier_agent, subtracter_agent, divider_agent, number_agent],
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
    messages=[],
    max_round=12,
    send_introductions=True,
)

constrained_group_chat_manager = GroupChatManager(
    groupchat=constrained_graph_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

chat_result = number_agent.initiate_chat(
    constrained_group_chat_manager,
    message="My number is 3, I want to turn it into 10. Once I get to 10, keep it there.",
    summary_method="reflection_with_llm",
)

Number_Agent (to chat_manager):

My number is 3, I want to turn it into 10. Once I get to 10, keep it there.

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

4

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

8

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

7

--------------------------------------------------------------------------------
Divider_Agent (to chat_manager):

3.5

--------------------------------------------------------------------------------
Adder_Agent (to chat_manager):

4.5

--------------------------------------------------------------------------------
Multiplier_Agent (to chat_manager):

9

--------------------------------------------------------------------------------
Subtracter_Agent (to chat_manager):

8

----------------------------------------------------

This time, the agents are selected following the constraints we have specified.

## Nested Chat

[To be completed...]